# Tutorial

In [1]:
import gpu_tracker.tracker as track
import time

The `gpu_tracker.tracker` module provides the `Tracker` class which uses an underlying thread to measure computational resource usage, namely the compute time, maximum RAM used, and maximum GPU memory used. The `start()` method starts this thread which tracks usage in the background. After calling `start()`, write the code to measure resource usage, followed by calling the `stop()` method. The compute time will be the time from the call to `start()` to the call to `stop()` and the RAM and GPU quantities will be the amount of memory used by the code that's in between `start()` and `stop()`. The `Tracker` class additionally has a `__str__` method so it can be printed as a string that formats the values and units of each computational resource.

In [2]:
tracker = track.Tracker()
tracker.start()
# Perform expensive operations
time.sleep(5)
tracker.stop()
print(tracker)

Max RAM: 0.068 gigabytes
Max GPU: 0.000 gigabytes
Compute time: 0.001 hours


The equivalent can be accomplished using `Tracker` as a context manager rather than explicitly calling `start()` and `stop()`.

In [3]:
with track.Tracker() as tracker:
    # Perform expensive operations
    time.sleep(5)
print(tracker)

Max RAM: 0.068 gigabytes
Max GPU: 0.000 gigabytes
Compute time: 0.001 hours


The units of the computational resources can be modified as desired. For example, to measure the RAM in megabytes, the GPU memory in kilobytes, and the compute time in seconds:

In [4]:
with track.Tracker(ram_unit='megabytes', gpu_unit='kilobytes', time_unit='seconds') as tracker:
    # Perform expensive operations
    time.sleep(3)
print(tracker)

Max RAM: 67.662 megabytes
Max GPU: 0.000 kilobytes
Compute time: 3.112 seconds


Additionally, the individual measurements and units are available as attributes in the `Tracker` class.

In [5]:
print(tracker.max_ram, tracker.ram_unit)

67.661824 megabytes
